Final.ipynb file should contain the following :

It should have two functions.

Function-1

Should include entire pipeline, from data preprocessing to making final predictions.

It should take in raw data as input.

It should return predictions for your input. Here the input can be a single point or a set of points.

def final_fun_1(X):

.....

..... # you will use the best model that you found out with your experiments
return predictions made on X ( Raw Data)

Function-2

Should include entire pipeline, from data preprocessing to making final predictions.

It should take in raw data as input along with its target values.

It should return the metric value that you are judging your models on.

def final_fun_2(X,Y):
.....

..... # you will use the best model that you found out with your experiments
return final_metric computed on X ( Raw Data) and Y (target variable)

# 1. Importing Libraries


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
from dateutil.parser import parse
import pickle
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from bs4 import BeautifulSoup
import pickle
from scipy.sparse import hstack
from tqdm import tqdm
import os
import pickle
from scipy.sparse import csr_matrix
from sklearn.metrics import accuracy_score, confusion_matrix,roc_auc_score, f1_score,roc_curve,log_loss
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.models import load_model


# Function1

In [2]:
with open('/content/drive/MyDrive/LSTM/featured_data','rb') as f:
  data=pickle.load(f)
X=data.drop('score',axis=1)
y=data['score']

In [3]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

# we are removing the words from the stop words list: 'no', 'nor', 'not'
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

def preprocess_text(text_data):
    #stemmer=SnowballStemmer(language="english")
    preprocessed_text = []
    # tqdm is for printing the status bar
    for sentance in text_data:
        sentance = re.sub(r"http\S+", "", sentance)
        sentance = BeautifulSoup(sentance, 'lxml').get_text()
        sentance = re.sub("\S*\d\S*", "", sentance).strip()
        sent = decontracted(sentance)
        sent = sent.replace('\\r', ' ')
        sent = sent.replace('\\n', ' ')
        sent = sent.replace('\\"', ' ')
        sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
        # https://gist.github.com/sebleier/554280
        sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
        sent=sent.lower().strip()
        #sent=" ".join(stemmer.stem(e) for e in sent.split())
        preprocessed_text.append(sent)
    return preprocessed_text


In [4]:
def final_func_1(X):
  """This function will preprocess all  data and 
  return prediction for a individual point
  X: individual point
  """
  #Dropping all the features which are having low variance and zero variance
  file=open('/content/drive/MyDrive/LSTM/drop_features','rb')
  drop_column=pickle.load(file)
  file.close()
  X.drop(drop_column,axis=1,inplace=True)
  X['Review']=X['ReviewTitle']+" "+X['CompleteReview']
  X.drop(['ReviewTitle','CompleteReview'],axis=1,inplace=True)
  X["process_review"]=preprocess_text(X["Review"].values)
  X.drop('Review',axis=1,inplace=True)
  with open('/content/drive/MyDrive/LSTM/vectorizer','rb') as f:
    vectorizer=pickle.load(f)
  X_bow=vectorizer.transform(X['process_review'].values)
  X_bow1 = hstack((X_bow,X.drop('process_review',axis=1).values)).tocsr()
  model=load_model('/content/drive/MyDrive/LSTM/weights-22-0.9548.hdf5')
  y_pred=model.predict(X_bow1)
  if y_pred>=0.5:
    y_class=1
  else:
    y_class=0
  return y_class



# Now test this function with some Data points

In [5]:
X.loc[[0]]['ReviewTitle'].values

array(['Good Company for Every employee'], dtype=object)

In [6]:
X.loc[[0]]['CompleteReview'].values

array(['Good company for every Engineers dream, Full Mediclaim for entired family, Free transport services from company location to home, Township culture for employees,job security.'],
      dtype=object)

**The above review is positive. Lets test this review with our model and predict the results**

In [7]:
x=X.loc[[0]]
y_pred=final_func_1(x)
print("The Class of the given review is ",y_pred)

The Class of the given review is  1


**Our model predicted correctly** 

*Now let's look at negative review*

In [8]:
X.loc[[4]]['ReviewTitle'].values

array(['Not good'], dtype=object)

In [9]:
X.loc[[4]]['CompleteReview'].values

array(['Work culture not good and no benefits. I would recommend anyone to join Mphasis atleast not in India. There is no growth in career and salary is too low. Also the HR are no help at all'],
      dtype=object)

**Now lets this review with our model**

In [10]:
x=X.loc[[4]]
y_pred=final_func_1(x)
print("The Class of the given review is ",y_pred)

The Class of the given review is  0


**Our model predicted correctly** 

# Function2

In [11]:
def final_func_2(X,y):
  """This function will preprocess all the data and 
  return Metric score(R-squared value) for a set of points"""
   #Dropping all the features which are having low variance and zero variance
  #Dropping all the features which are having low variance and zero variance
  file=open('/content/drive/MyDrive/LSTM/drop_features','rb')
  drop_column=pickle.load(file)
  file.close()
  X.drop(drop_column,axis=1,inplace=True)
  X['Review']=X['ReviewTitle']+" "+X['CompleteReview']
  X.drop(['ReviewTitle','CompleteReview'],axis=1,inplace=True)
  X["process_review"]=preprocess_text(X["Review"].values)
  X.drop('Review',axis=1,inplace=True)
  with open('/content/drive/MyDrive/LSTM/vectorizer','rb') as f:
    vectorizer=pickle.load(f)
  X_bow=vectorizer.transform(X['process_review'].values)
  X_bow1 = hstack((X_bow,X.drop('process_review',axis=1).values)).tocsr()
  model=load_model('/content/drive/MyDrive/LSTM/weights-22-0.9548.hdf5')
  y_pred=model.predict(X_bow1)
  y_class=[]
  for i in y_pred:
    if i>=0.5:
      y_class.append(1)
    else:
      y_class.append(0)
  accuracy=accuracy_score(y,y_class)
  f1_score_model=f1_score(y,y_class)
  auc=roc_auc_score(y,y_class)
  logloss=log_loss(y,y_class)
  return {"accuracy":accuracy,"f1_score":f1_score_model,"Auc_score":auc,"log_loss":logloss}


**Let's check the metrics for the whole data**

In [12]:
final_func_2(X,y)

{'Auc_score': 0.7180968349750804,
 'accuracy': 0.9575730003875381,
 'f1_score': 0.9776636417666835,
 'log_loss': 1.46540593586075}

**Accuracy achieved is 95.75%, auc_score is 0.72, f1_score is 0.97 and log_loss is 1.46**